# Initial Analysis of King County Housing Dataset

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
from random import gauss
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

In [3]:
# Reading in .csv, assigning to dataframe `df`
df = pd.read_csv('../data/kc_house_data.csv')

# Shows all columns, i.e. forces pandas/Jupyter
# not to truncate dataframe horizontally
pd.set_option('display.max_columns', False)

In [4]:
# Checking out the data
df.head(10);

In [5]:
# Checking out the data types and null/non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [13]:
# Converting column `date` to datetime64[ns] from type string/object
df['date'] = df['date'].astype('datetime64[ns]')

# trial = df.groupby(['id']).date.transform(max)
# trial_cleaned = df[df.id == trial]

# trial_cleaned = None

In [6]:
# Creating a straightforward, initial model --
# just for kicks
simple_formula = 'price ~ sqft_living'
simple_model = ols(formula = simple_formula,
                   data = df).fit()

In [25]:
numbers_df = df.select_dtypes('number')

numbers_df.head()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,221900.0,3,1.00,1180,5650,1.0,1180,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,538000.0,3,2.25,2570,7242,2.0,2170,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,180000.0,2,1.00,770,10000,1.0,770,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,604000.0,4,3.00,1960,5000,1.0,1050,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,510000.0,3,2.00,1680,8080,1.0,1680,1987,0.0,98074,47.6168,-122.045,1800,7503


In [29]:
df_copy2 = df.sort_values(by=['id', 'date'])
df_copy2 = df_copy2.drop_duplicates(subset=['id'], keep='last')

df_copy2;

(21420, 21)

### Initial thoughts

#### The Good
- How should we approach `sqft_living` vs. `sqft_above`? What, if anything, is the difference between these two metrics? `sqft_above` counts living area separate from the basement, but it seems the parameter of interest there is whether or not the house *has* livable basement area.
    - Running a heat map proves that these variables have a colinearity coefficient of `0.88`; it's likely we'll need to choose one or the other for a linear model.
- Because `sqft_living` refers to a record and `sqft_living15` refers to properties in the geographical vicinity of that record, we can compare the values stored in these two columns in a given row to determine if a home's square footage is greater than or less than nearby homes.
    - It might be useful to, early on, perform some simple linear regressions using these as predictor variables to determine goodness of fit, etc.
    
#### The Less Good
- Is `id` going to be useful in any way? It doesn't seem like an instance where a 'unique identifier' is going to provide us with any information we can use for prediction.
    - However! - see "Notes for Cleaning" below.
- We are working with data from a *single county* - will `lat` and `long` be able to tell us anything that we can't glean from, say, zip code? This might also make our presentation and recommendations unnecessarily complex.

#### The... Not Sure?
- `yr_renovated` has a fairly substantial number of null values (~4,000), and an even greater number of records with value `0.0`, which likely indicates that no renovation has been done to the house (need to check data dictionary).
    - Heat map examination also indicates a low correlation, < 0.1, between `yr_built` and `price`.   
- `waterfront` may be useful as a boolean value, i.e. to test whether homes located on water are higher priced than homes that we **know** are not located on water.

#### Notes for Cleaning
- Coded null values exist in the following columns:
    - `waterfront`
    - `view`
    - `yr_renovated`
- Duplicates exist in the `id` column - these are houses that were sold more than once! What are we going to do about this, and why are we going to go about it that way?

In [6]:
# Dealing with duplicates in the `id` column, keeping only the most recent sale

## code goes here

In [7]:
# Some exploratory value counts on columns of interest

# Counting the number of homes with recorded
# waterfront status vs. those without it
df.waterfront.value_counts()


# Number of bathrooms - note that many values exist
# with decimal places, i.e. half baths. This could get
# messy and convoluted quickly.

## df.bathrooms.value_counts()

NO     19075
YES      146
Name: waterfront, dtype: int64

In [8]:
# Filling null values in column `view` with string indicating
# that no record exists on the property's view
df.view.fillna(value = 'NO RECORD', inplace = True)

# Filling null values in column `yr_renovated` with integer 0,
# indicating that no renovation year exists for this record
df.yr_renovated.fillna(value = 0, inplace = True)

# Filling null values in column `waterfront` to reflect
# unknown status - this code may go unused
df.waterfront.fillna(value = 'UNKNOWN', inplace = True)

In [9]:
# Creating variables to count values in columns `grade`, `condition`, 
# and `yr_renovated` in King County real estate data
grade_counts = df['grade'].value_counts()
condition_counts = df['condition'].value_counts()
renovation_counts = df['yr_renovated'].value_counts()

# Printing value counts
print(f'Condition value counts:\n{condition_counts}\n')
print(f'Grade value counts:\n{grade_counts}\n')
print(f'Renovation year value counts:\n{renovation_counts}\n')

Condition value counts:
Average      14020
Good          5677
Very Good     1701
Fair           170
Poor            29
Name: condition, dtype: int64

Grade value counts:
7 Average        8974
8 Good           6065
9 Better         2615
6 Low Average    2038
10 Very Good     1134
11 Excellent      399
5 Fair            242
12 Luxury          89
4 Low              27
13 Mansion         13
3 Poor              1
Name: grade, dtype: int64

Renovation year value counts:
0.0       20853
2014.0       73
2003.0       31
2013.0       31
2007.0       30
          ...  
1946.0        1
1959.0        1
1971.0        1
1951.0        1
1954.0        1
Name: yr_renovated, Length: 70, dtype: int64



In [10]:
# Dropping columns we determined were either superfluous or irrelevant
df.drop(columns = ['id', 'sqft_lot', 'floors', 'condition', 'sqft_above', 'sqft_basement',
                   'zipcode', 'lat', 'long', 'sqft_lot15'], inplace = True)
# potentially include yr_renovated? depending on use

In [11]:
# Applied universally to notebook -- converts any scientific notation
# to standard notation, rounded to two decimal places

# Use with caution! Will need to restart kernel to reset effect.

## pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [12]:
# Creating new df, grouped by column `view`, and looking
# at mean values for the categorical variables in `view`
view_df = df.groupby('view').mean()

view_df.sort_values('price', ascending = False)

,price,bedrooms,bathrooms,sqft_living,yr_built,yr_renovated,sqft_living15
view,,,,,,,
EXCELLENT,1.452466e+06,3.624606,2.782334,3334.479495,1965.949527,270.179811,2841.000000
GOOD,9.732852e+05,3.671260,2.670276,3016.846457,1967.389764,192.582677,2702.208661
FAIR,8.133733e+05,3.536364,2.350000,2571.045455,1962.890909,151.109091,2407.345455
AVERAGE,7.913904e+05,3.567398,2.426594,2650.719958,1964.817137,106.271682,2427.344828
NO RECORD,6.219582e+05,3.428571,2.154762,2249.174603,1970.111111,31.666667,2096.126984
NONE,4.968061e+05,3.348780,2.071028,1998.364793,1971.621821,59.104572,1924.738492


Takeaways from this quick analysis of `view` include...

- Clear descending order with `EXCELLENT` at top and `NONE` at bottom
- Significant differences between `EXCELLENT` and middle `view` values, and between middle `view` values and `NONE`
    - Differences not as significant between `GOOD`, `FAIR`, and `AVERAGE`
- `NO RECORD` (formerly null/NaN) will need further investigation, as it seems to lie between `AVERAGE` (i.e. **having** a view, but an average one) and `NONE`, which ostensibly means no scenic view whatsoever.
    - Fortunately, we have < 100 null values in this column, so we can either drop them or modify them without worrying about those changes affecting our conclusions significantly.

### Side notes and things to revisit

- `yr_renovated` has null values and a lot of `0` values too. Of the houses *with* values in the `yr_renovated` column, the vast majority were renovations from 2013 and 2014. It might be worth further exploring how these recently renovated homes compare pricewise to homes built in prior years.